In [4]:
import sys
import warnings
warnings.filterwarnings('ignore')

# import local modules
sys.path.append('../src')
from Fishnet import Fishnet

# update files automatically
%load_ext autoreload
%autoreload 2

## Install package within conda using tensorflow
## /opt/homebrew/Caskroom/miniconda/base/envs/capstone_2/bin/pip install tensorflow

# Dataset Generation

In [16]:
FISHNET_PATH = "../Gis/Fishnet/fishnet_texas_small.pkl" #'../Gis/Fishnet/fishnet_quarter_mile_completed_2023-07-13.pkl' #
SAVE_PATH = '../Results/'
FILE_NAME = 'urbanization_data_small_texas' 

In [17]:
fc = Fishnet.load(FISHNET_PATH)

# Compute Urbanization Rate
years = list(range(2016, 2023))
for yr in years[1:]:
    fc.compute_difference(
        f"MeanPixel_{yr}", f"MeanPixel_{yr-1}", filtered=True, normalize=True
    )

# rename MeanPixel_2017-MeanPixel_2016 to urbanization_rate_2016
for yr in years[1:]:
    fc.filtered_fishnet.rename(
        columns={f"MeanPixel_{yr}-MeanPixel_{yr-1}": f"urbanization_rate_{yr}"},
        inplace=True,
    )

for yr in years:
    fc.filtered_fishnet.rename(
        columns={f"MeanPixel_{yr}": f"urbanization_{yr}"}, inplace=True
    )

fc.filtered_fishnet.rename(columns={"id": "tile_id"}, inplace=True)
fc.filtered_fishnet["tile_id"] = fc.filtered_fishnet["tile_id"].astype(int)
fc.filtered_fishnet["batch_id"] = fc.filtered_fishnet["batch_id"].astype(int)

# Extracting Lat Long coordinates
fc.filtered_fishnet['centroid'] = fc.filtered_fishnet['geometry'].apply(lambda x: x.centroid)
fc.filtered_fishnet['Lat'] = fc.filtered_fishnet['centroid'].apply(lambda x: x.y)
fc.filtered_fishnet['Lon'] = fc.filtered_fishnet['centroid'].apply(lambda x: x.x)

In [18]:
vars1 = ["tile_id", "batch_id"] + [f"urbanization_rate_{year}" for year in range(2017, 2023)]
vars2 = ["tile_id", "batch_id"] + [f"urbanization_{year}" for year in range(2016, 2023)]

data = fc.filtered_fishnet[vars1].melt(
    id_vars=["tile_id", "batch_id"], var_name="year", value_name="urbanization_rate"
)
data["year"] = data["year"].str[-4:]
data["urbanization"] = fc.filtered_fishnet[vars2].melt(
    id_vars=["tile_id", "batch_id"], var_name="year", value_name="urbanization"
)["urbanization"] / 255

# data['Lat'] is the latitude of the centroid of the tile in fc.filtered_fishnet['Lat'] joint
data = data.merge(fc.filtered_fishnet[["tile_id", "batch_id", "Lat", "Lon"]], on=["tile_id", "batch_id"])

data.head()

,tile_id,batch_id,year,urbanization_rate,urbanization,Lat,Lon
0,6060154,23743,2017,0.063333,0.710556,29.793894,-95.801855
1,6060154,23743,2018,0.097778,0.773889,29.793894,-95.801855
2,6060154,23743,2019,-0.010556,0.871667,29.793894,-95.801855
3,6060154,23743,2020,0.025000,0.861111,29.793894,-95.801855
4,6060154,23743,2021,0.027778,0.886111,29.793894,-95.801855


# Save dataframe and metadata

In [19]:
# Save CSV
data.to_csv(SAVE_PATH + FILE_NAME + ".csv", index=False)

In [20]:
# Save Metadata
with open(SAVE_PATH + FILE_NAME + '.txt', 'w') as f:
    f.write("\n\nGeneral Fishnet Information:\n")
    f.write(str(fc.fishnet_info(return_=True))+"\n\n")
    f.write("\n\nGeneral Batch Information:\n")
    f.write(str(fc.batch_info(return_=True)))
    if fc.filtered:
        f.write("\n\n\nGeneral Filter Information:")
        f.write("\nFiltered region: " + str(fc.filter_region))
        f.write("\nNumber of rows: " + str(fc.filtered_fishnet_rows))
        f.write("\nNumber of cols: " + str(fc.filtered_fishnet_cols))


 Fishnet Object has the following attributes: 

+----------------+-----------------------+-------------------+
|     Metric     |        Degrees        |       Miles       |
+----------------+-----------------------+-------------------+
| Fishnet Width  |   13.138423211000031  | 817.0318617424371 |
| Fishnet Height |   10.663161770999977  |  736.75306466581  |
|  Tiles Width   |  0.004020157322514706 |        0.25       |
|  Tiles Height  | 0.0036182855831725647 |        0.25       |
+----------------+-----------------------+-------------------+

Fishnet Batch has the following attributes: 

+---------+---------+---------+
|  Metric |  Tiles  | Batches |
+---------+---------+---------+
|   Rows  |   2948  |   185   |
| Columns |   3269  |   205   |
|  Cells  | 9637012 |  37925  |
+---------+---------+---------+
